### **Import Library & Load Data**



In [ ]:
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
from tabulate import tabulate
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

def read_file(nama_file):
  data=pd.read_csv(nama_file)
  return data

nama_file = ("Dataset PKA.csv")
data = read_file(nama_file)
data.head()

,Bahan_Makanan,Kalori_(kkal),Karbohidrat_(gram),Protein_(gram),Lemak_(gram),Harga,Keterangan
0,Alpukat,85,7.7,0.9,6.5,7500,100 gram
1,Apel,58,14.9,0.3,0.4,4500,100 gram
2,Air kelapa muda,46,8.9,1.7,0.5,6250,250 ml
3,Dada ayam,302,0.0,18.2,25.0,6000,100 gram
4,Belimbing,36,8.8,0.4,0.4,2450,100 gram


In [ ]:
from re import L
BM = data['Bahan_Makanan']
Kalori = data['Kalori_(kkal)']
Karbo = data['Karbohidrat_(gram)']
Lemak = data['Lemak_(gram)']
Protein = data['Protein_(gram)']
Harga = data['Harga']

In [ ]:
a = pd.DataFrame({'Bahan Makanan':BM})
b = pd.DataFrame({'Kalori':Kalori})
c = pd.DataFrame({'Karbohidrat':Karbo})
d = pd.DataFrame({'Lemak':Lemak})
e = pd.DataFrame({'Protein':Protein})
f = pd.DataFrame({'Harga':Harga})

pdList = [a,b,c,d,e,f,]
df = pd.concat(pdList,axis=1)
df.head()

,Bahan Makanan,Kalori,Karbohidrat,Lemak,Protein,Harga
0,Alpukat,85,7.7,6.5,0.9,7500
1,Apel,58,14.9,0.4,0.3,4500
2,Air kelapa muda,46,8.9,0.5,1.7,6250
3,Dada ayam,302,0.0,25.0,18.2,6000
4,Belimbing,36,8.8,0.4,0.4,2450


### **Greedy Algorithm**

Batas Kandungan Nutrisi

In [ ]:
kal_total = 2000
karbo_total = 50
lemak_total = (kal_total*0.75)/9
pro_total = (kal_total*0.2)/4

Menghitung rasio lemak terhadap karbohidrat dan protein

In [ ]:
#Inisialisasi alpha dan beta
alpha = 0.7
beta = 0.3

In [ ]:
fat_to_carb_ratio = round(df['Lemak'] / df['Karbohidrat'],2)
fat_to_pro_ratio = round(df['Lemak'] / df['Protein'],2)
score1 = round(alpha*fat_to_pro_ratio + beta*fat_to_carb_ratio,2)

#Menambahkan kolom pada data frame
df['lemak_per_karbohidrat'] = fat_to_carb_ratio
df['lemak_per_protein'] = fat_to_pro_ratio
df['Score_1'] = score1

#Mengurutkan data
sorted_data_1 = df.sort_values(by='Score_1', ascending=False)
sorted_data_1.head()

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Bahan Makanan,Kalori,Karbohidrat,Lemak,Protein,Harga,lemak_per_karbohidrat,lemak_per_protein,Score_1
15,Ikan mujair,89,0.0,1.0,18.7,4500,inf,0.05,inf
3,Dada ayam,302,0.0,25.0,18.2,6000,inf,1.37,inf
61,Minyak kelapa,870,0.0,98.0,1.0,7000,inf,98.00,inf
11,Daging sapi,207,0.0,14.0,18.8,8900,inf,0.74,inf
13,Hati ayam,116,0.0,4.8,16.9,4500,inf,0.28,inf


In [ ]:
bahan_terpilih =[]
total_kal = 0
total_karbo = 0
total_lemak = 0
total_pro = 0
total_harga = 0
anggaran = input('Masukan batas harga : ')
anggaran = int(anggaran)

for index, bahan in sorted_data_1.iterrows():
  if total_lemak + bahan['Lemak'] <= lemak_total and total_pro + bahan['Protein']<=pro_total and total_karbo + bahan['Karbohidrat']<=karbo_total and total_harga + bahan['Harga']<=anggaran:
    bahan_terpilih.append(bahan['Bahan Makanan'])
    total_kal += bahan['Kalori']
    total_lemak += bahan['Lemak']
    total_pro += bahan['Protein']
    total_karbo += bahan['Karbohidrat']
    total_harga += bahan['Harga']

data_tabel1=[]
bahan_string = ', '.join(bahan_terpilih)
data_tabel1.append([np.transpose(bahan_string), total_kal, total_lemak, total_pro, total_karbo, total_harga])
header = ['Bahan Makanan', 'Total Kalori', 'Total Lemak', 'Total Protein', 'Total Karbohidrat', 'Total Harga']
print(tabulate(data_tabel1, headers=header, tablefmt='grid'))

Masukan batas harga : 100000
+-----------------------------------------------------------------------------------------------------------------------+----------------+---------------+-----------------+---------------------+---------------+
| Bahan Makanan                                                                                                         |   Total Kalori |   Total Lemak |   Total Protein |   Total Karbohidrat |   Total Harga |
+=======================================================================================================================+================+===============+=================+=====================+===============+
| Ikan mujair, Dada ayam, Minyak kelapa, Daging sapi, Hati ayam , Ikan kembung, Pepaya, Alpukat, Lemon, Apel, Belimbing |           1946 |         163.9 |            97.9 |                49.8 |         54150 |
+-----------------------------------------------------------------------------------------------------------------------+------

Menghitung rasio lemak dan karbohidrat terhadap kalori

In [ ]:
fat_to_cal_ratio = round(df['Lemak'] / df['Kalori'],2)
carb_to_cal_ratio = round(df['Karbohidrat'] / df['Kalori'],2)
score2 = round(alpha*fat_to_cal_ratio + beta*carb_to_cal_ratio,2)

#Menambahkan kolom pada data frame
df['lemak_per_kalori'] = fat_to_cal_ratio
df['karbo_per_kalori'] = carb_to_cal_ratio
df['Score_2'] = score2

#Mengurutkan data
sorted_data_2 = df.sort_values(by='Score_2', ascending=False)
sorted_data_2.head()
#print(sorted_data_2)

,Bahan Makanan,Kalori,Karbohidrat,Lemak,Protein,Harga,lemak_per_karbohidrat,lemak_per_protein,Score_1,lemak_per_kalori,karbo_per_kalori,Score_2
42,Pepaya,46,12.2,12.0,0.2,800,0.98,60.00,42.29,0.26,0.27,0.26
59,Talas,98,1.9,23.7,0.2,1500,12.47,118.50,86.69,0.24,0.02,0.17
41,Nasi Beras merah,149,2.8,32.5,0.4,3000,11.61,81.25,60.36,0.22,0.02,0.16
29,Kenari,657,13.0,66.0,15.0,11800,5.08,4.40,4.60,0.10,0.02,0.08
1,Apel,58,14.9,0.4,0.3,4500,0.03,1.33,0.94,0.01,0.26,0.08


In [ ]:
bahan_terpilih =[]
total_kal = 0
total_karbo = 0
total_lemak = 0
total_pro = 0
total_harga = 0
anggaran = input('Masukan batas harga : ')
anggaran = int(anggaran)

for index, bahan in sorted_data_2.iterrows():
  if total_lemak + bahan['Lemak'] <= lemak_total and total_kal + bahan['Kalori']<=kal_total and total_karbo + bahan['Karbohidrat']<=karbo_total and total_harga + bahan['Harga']<=anggaran:
    bahan_terpilih.append(bahan['Bahan Makanan'])
    total_kal += bahan['Kalori']
    total_lemak += bahan['Lemak']
    total_pro += bahan['Protein']
    total_karbo += bahan['Karbohidrat']
    total_harga += bahan['Harga']

data_tabel2=[]
bahan_string = ', '.join(bahan_terpilih)
data_tabel2.append([np.transpose(bahan_string), total_kal, total_lemak, total_pro, total_karbo, total_harga])
header = ['Bahan Makanan', 'Total Kalori', 'Total Lemak', 'Total Protein', 'Total Karbohidrat', 'Total Harga']
print(tabulate(data_tabel2, headers=header, tablefmt='grid'))

Masukan batas harga : 100000
+---------------------------------------------------------------------------------------------------------------------+----------------+---------------+-----------------+---------------------+---------------+
| Bahan Makanan                                                                                                       |   Total Kalori |   Total Lemak |   Total Protein |   Total Karbohidrat |   Total Harga |
+=====================================================================================================================+================+===============+=================+=====================+===============+
| Pepaya, Talas, Nasi Beras merah, Kenari, Apel, Susu kedelai, Dada ayam, Cumi cumi, Ikan mujair, Ikan kembung, Udang |           1709 |           165 |           115.6 |                  50 |         58600 |
+---------------------------------------------------------------------------------------------------------------------+----------------

Menghitung rasio lemak dan protein tergadap harga (Profit)

In [ ]:
alpha1 = 0.7
beta1 = 0.2
theta1 = 0.1
fat_to_price_ratio = round(df['Lemak'] / df['Harga'],2)
pro_to_price_ratio = round(df['Protein'] / df['Harga'],2)
carb_to_price_ratio = round(df['Karbohidrat'] / df['Harga'],2)
score3 = round(alpha1*fat_to_price_ratio + beta1*pro_to_price_ratio - theta1*carb_to_price_ratio,2)

#Menambahkan kolom pada data frame
df['lemak_per_harga'] = fat_to_price_ratio
df['protein_per_harga'] = pro_to_price_ratio
df['karbo_per_harga'] = carb_to_price_ratio
df['Score_3'] = score3

#Mengurutkan data
sorted_data_3 = df.sort_values(by='Score_3', ascending=False)
sorted_data_3.tail()
#print(sorted_data_2)

,Bahan Makanan,Kalori,Karbohidrat,Lemak,Protein,Harga,lemak_per_karbohidrat,lemak_per_protein,Score_1,lemak_per_harga,protein_per_harga,karbo_per_harga,Score_3
26,Kangkung,29,5.4,0.3,3.0,2000,0.06,0.10,0.09,0.0,0.0,0.00,0.0
27,Keju,326,13.1,20.3,22.8,12000,1.55,0.89,1.09,0.0,0.0,0.00,0.0
28,Kentang,83,19.1,0.1,2.0,1500,0.01,0.05,0.04,0.0,0.0,0.01,0.0
30,Kepiting,151,14.1,3.8,13.8,15000,0.27,0.28,0.28,0.0,0.0,0.00,0.0
64,Tomat,24,4.7,0.5,1.3,1200,0.11,0.38,0.30,0.0,0.0,0.00,0.0


In [14]:
bahan_terpilih =[]
total_kal = 0
total_karbo = 0
total_lemak = 0
total_pro = 0
total_harga = 0
anggaran = input('Masukan batas harga : ')
anggaran = int(anggaran)

for index, bahan in sorted_data_3.iterrows():
  if total_lemak + bahan['Lemak'] <= lemak_total and total_pro + bahan['Protein']<=pro_total and total_karbo + bahan['Karbohidrat']<=karbo_total and total_harga + bahan['Harga']<=anggaran:
    bahan_terpilih.append(bahan['Bahan Makanan'])
    total_kal += bahan['Kalori']
    total_lemak += bahan['Lemak']
    total_pro += bahan['Protein']
    total_karbo += bahan['Karbohidrat']
    total_harga += bahan['Harga']

data_tabel3=[]
bahan_string = ', '.join(bahan_terpilih)
data_tabel3.append([np.transpose(bahan_string), total_kal, total_lemak, total_pro, total_karbo, total_harga])
header = ['Bahan Makanan', 'Total Kalori', 'Total Lemak', 'Total Protein', 'Total Karbohidrat', 'Total Harga']
print(tabulate(data_tabel3, headers=header, tablefmt='grid'))

Masukan batas harga : 10000
+-----------------+----------------+---------------+-----------------+---------------------+---------------+
| Bahan Makanan   |   Total Kalori |   Total Lemak |   Total Protein |   Total Karbohidrat |   Total Harga |
+=================+================+===============+=================+=====================+===============+
| Wijen           |            568 |          51.1 |            19.3 |                18.1 |          9900 |
+-----------------+----------------+---------------+-----------------+---------------------+---------------+
